In [1]:
import nltk
from nltk import word_tokenize, pos_tag
import pprint 

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
print (pos_tag(word_tokenize("I'm learning NLP"))) # print tag of every token

[('I', 'PRP'), ("'m", 'VBP'), ('learning', 'VBG'), ('NLP', 'NNP')]


In [2]:
tagged_sentences = nltk.corpus.treebank.tagged_sents() # use treebank corpus for tagged sentences
 
print (tagged_sentences[0])
print ("Tagged sentences: ", len(tagged_sentences))
print ("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


In [4]:
# function to create features
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }
 
pprint.pprint(features(['This', 'is', 'a', 'sentence'], 2))
 

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}


In [5]:
# helper function to strip the tags from tagged corpus
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [6]:
# Split the dataset for training and testing
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]
 
print ('Training sentences {}'.format(len(training_sentences)))   # 2935
print ('Test sentences {}'.format(len(test_sentences)))         # 979

Training sentences 2935
Test sentences 979


In [7]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(training_sentences)

In [8]:
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:10000], y[:10000])   # Use only the first 10K samples, for faster training
 
print ('Training completed')
 
X_test, y_test = transform_to_dataset(test_sentences)
 
print ("Accuracy:", clf.score(X_test, y_test))

Training completed
Accuracy: 0.895870159087257


In [9]:
# helper function to tagged unseeen sentences
def pos_tag(sentence):
    tagged_sentence = []
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

In [11]:
[(i,j) for (i,j) in pos_tag(word_tokenize('Machine learning is awesome'))]

[('Machine', 'RB'), ('learning', 'VBG'), ('is', 'VBZ'), ('awesome', 'NN')]

In [12]:
[(i,j) for (i,j) in pos_tag(word_tokenize('I love NLP and all the possibilities it has'))]

[('I', 'IN'),
 ('love', 'JJ'),
 ('NLP', 'NNP'),
 ('and', 'CC'),
 ('all', 'DT'),
 ('the', 'DT'),
 ('possibilities', 'NNS'),
 ('it', 'PRP'),
 ('has', 'VBZ')]